In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from datasets import load_dataset, load_metric

alocinit2.log
atamptl_log_ratelimit.txt
code-d54a4ff1-1b4b-4eb2-9f06-e189aa720a9a=
hsperfdata_root/
model.gin_model_GINConv_propagate_rozpk9q2.py
outdated_cache_ogb
outdated_cache_outdated
__pycache__/
pymp-09t_n7rr/
pymp-2awtn60i/
pymp-bpc5u3gn/
pymp-fwjbx4jr/
pymp-gqlgfl58/
pymp-q1c0h2hp/
pymp-uv9uahli/
pymp-vlod0emx/
pyright-1081498-5xlqnTkRVicY/
pyright-1081498-TrhbOEWHX1O4/
pytorch-errorfile-m558rrps.pickle
snap-private-tmp/
systemd-private-a819b4bb1cf54fec950021f1a2b5d5cc-systemd-logind.service-pzisHK/
systemd-private-a819b4bb1cf54fec950021f1a2b5d5cc-systemd-oomd.service-X51Y5E/
systemd-private-a819b4bb1cf54fec950021f1a2b5d5cc-systemd-resolved.service-of5urb/
systemd-private-a819b4bb1cf54fec950021f1a2b5d5cc-systemd-timesyncd.service-NFIx5T/
systemd-private-a819b4bb1cf54fec950021f1a2b5d5cc-upower.service-iEJHYr/
tmp1vv5xdtowandb-media/
tmp46wld5z_wandb-artifacts/
tmp6jpb7o16wandb/
tmpcy638vt8/
tmpiogjwa62wandb-media/
tmux-1002/
tmux-1009/
v3init2.log
vtmpsAQmp0x
vtmpsLCtHrv


In [8]:
# Load Evaluation Dataset
# Import Hugging Face Transformers and Datasets
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from datasets import load_dataset

# Load the model and tokenizer
# Check if CUDA is available and set the device

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("GT4SD/multitask-text-and-chemistry-t5-small-augm")
model = AutoModelForSeq2SeqLM.from_pretrained("GT4SD/multitask-text-and-chemistry-t5-small-augm", device_map=device)

# Load and preprocess the dataset
dataset = load_dataset("language-plus-molecules/LPM-24_train")



# Move the model to the specified device
model.to(device)

# Tokenize the dataset

""" def tokenize_function(examples):
    return tokenizer(examples["molecule"], padding="max_length", truncation=True) """

""" def tokenize_function(examples):
    # Add prompt
    prompt = "Caption the following SMILES: "
    examples['molecule'] = prompt + examples['molecule']
    model_inputs = tokenizer(examples["molecule"], padding="max_length", truncation=True)
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["caption"], padding="max_length", truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs """

def tokenize_function(examples):
    # Add prompt
    prompt = "Caption the following SMILES: "
    
    # Prepend the prompt to the 'molecule' field
    examples['molecule'] = [prompt + molecule for molecule in examples['molecule']]
    
    # Tokenize the input text
    model_inputs = tokenizer(examples["molecule"], padding="max_length", truncation=True)
    
    # Tokenize the target text
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["caption"], padding="max_length", truncation=True)
        
    # Add labels to the model inputs
    model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs


tokenized_datasets = dataset.map(tokenize_function, batched=True)

### Split train dataset into train and validation (85:15)
train_dataset = tokenized_datasets["split_train"].train_test_split(test_size=0.15).shuffle(seed=42)
print(train_dataset)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Map:   0%|          | 0/160560 [00:00<?, ? examples/s]/data/project/tsouthiratn/miniconda3/envs/FT5/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 33696/33696 [00:08<00:00, 37

DatasetDict({
    train: Dataset({
        features: ['molecule', 'caption', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 107834
    })
    test: Dataset({
        features: ['molecule', 'caption', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 19030
    })
})


In [9]:
# print the first example of the training dataset
print(train_dataset['train'][0]['molecule'])

Caption the following SMILES: CC/C=C\C/C=C\C/C=C\C/C=C\C/C=C\C/C=C\CCC(=O)OC[C@H](COP(=O)(O)OC[C@@H](O)COP(=O)(O)OC[C@@H](COC(=O)CCCCCCCCCCCCCCC)OC(=O)CCCCCCCCCCCCCCCCC)OC(=O)CCC/C=C\C/C=C\C/C=C\C/C=C\C/C=C\CC


In [10]:
print(dataset['train'][0]['molecule'])

CCCCCCCCCCCCCCCCCCCCCCCCC(=O)O[C@H](COC(=O)CCCCCCCCCCCCCCCCCCC)COP(=O)(O)OC[C@@H](O)COP(=O)(O)OC[C@@H](COC(=O)CCCCCCCCC(C)CC)OC(=O)CCCCCCCCCCCC(C)C


In [11]:
print(tokenized_datasets['split_train'][0]['molecule'])

Caption the following SMILES: CCCCCCCCCCCCCCCCCCCCCCCCC(=O)O[C@H](COC(=O)CCCCCCCCCCCCCCCCCCC)COP(=O)(O)OC[C@@H](O)COP(=O)(O)OC[C@@H](COC(=O)CCCCCCCCC(C)CC)OC(=O)CCCCCCCCCCCC(C)C
